In [0]:
dbutils.widgets.text(name="env",defaultValue="",label=" Enter the environment in lower case")
env = dbutils.widgets.get("env")

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, LongType, TimestampType, DateType

path = f"abfss://stock-project-container@bgardzinski1stock.dfs.core.windows.net/{env}"

input_path_pric = f"{path}/raw-data/hourly_prices"
schema_loc_pric = f"{path}/raw_data/_schema_autoloader/prices/"
checkpoint_loc_pric = f"{path}/Bronze/_checkpoints/prices_stream/"


def read_prices_stream():
    schema = StructType([
    StructField("symbol", StringType(), True),
    StructField("current_price", DoubleType(), True),
    StructField("open", DoubleType(), True),
    StructField("day_high", DoubleType(), True),
    StructField("day_low", DoubleType(), True),
    StructField("previous_close", DoubleType(), True),
    StructField("volume", DoubleType(), True),
    StructField("market_cap", DoubleType(), True),
    StructField("extract_time", TimestampType(), True),
    StructField("extract_date", DateType(), True),
    StructField("year", IntegerType(), True),  
    StructField("month", IntegerType(), True), 
    StructField("day", IntegerType(), True) 
])

    df = (
        spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("cloudFiles.schemaLocation", schema_loc_pric)
        .option("header", "true")
        .schema(schema)
        .load(input_path_pric)
    )

    return df

def write_prices(df):
    query = (
        df.writeStream \
    .format("delta") \
    .option("checkpointLocation", checkpoint_loc_pric) \
    .trigger(once=True) \
    .toTable(f"hive_metastore.{env}_bronze.prices") \
    .awaitTermination() )


df = read_prices_stream()
write_prices(df)